# **Summary Notebook `02` and `03`**

## **Part A: Environment & Setup**

* Imports, FrozenLake setup
* Initial policy definition (arbitrary)
* Helper functions (`compute_state_value`, `compute_q_value`)

In [2]:
# ==================================================================================================
# PART A: ENVIRONMENT SETUP & INITIAL DEFINITIONS
# ==================================================================================================
import gymnasium as gym

# Create FrozenLake environment
env = gym.make("FrozenLake-v1", is_slippery=True)

print("Number of states:", env.observation_space.n)   # 16
print("Number of actions:", env.action_space.n)       # 4

num_states = env.observation_space.n
num_actions = env.action_space.n
discount_factor = 0.9
terminal_state = num_states - 1   # assume last state is terminal

# Example initial policy (arbitrary)
policy = {
    0:1, 1:2, 2:1, 3:1,
    4:3, 5:1, 6:2, 7:3,
    8:0, 9:1, 10:2, 11:3,
    12:0, 13:1, 14:2, 15:3
}

# Running one episode with given policy (demo)
state, info = env.reset()
terminated = False
while not terminated:
    action = policy[state]
    next_state, reward, terminated, truncated, info = env.step(action)
    state = next_state

# Check transition probabilities from model
print(env.unwrapped.P[state][action])  
# Output format: (probability, next_state, reward, is_terminal)

# ======================================================================================
# Helpers for state-value and Q-value
# ======================================================================================
value = {state: 0 for state in range(num_states)}  # initial dummy values

def compute_state_value(state):
    if state == terminal_state:
        return 0
    action = policy[state]  # deterministic policy
    _, next_state, reward, _ = env.unwrapped.P[state][action][0]
    return reward + discount_factor * value[next_state]

def compute_q_value(state, action):
    if state == terminal_state:
        return None
    _, next_state, reward, _ = env.unwrapped.P[state][action][0]
    return reward + discount_factor * value[next_state]


# Now safely compute value function
value = {state: compute_state_value(state) for state in range(num_states)}
print("Initial Value Function:", value)

# Compute all Q-values
Q = {(state, action): compute_q_value(state, action) for state in range(num_states) for action in range(num_actions)}
print("Q-values:", Q)

Number of states: 16
Number of actions: 4
[(1.0, 5, 0, True)]
Initial Value Function: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0}
Q-values: {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0, (4, 0): 0.0, (4, 1): 0.0, (4, 2): 0.0, (4, 3): 0.0, (5, 0): 0.0, (5, 1): 0.0, (5, 2): 0.0, (5, 3): 0.0, (6, 0): 0.0, (6, 1): 0.0, (6, 2): 0.0, (6, 3): 0.0, (7, 0): 0.0, (7, 1): 0.0, (7, 2): 0.0, (7, 3): 0.0, (8, 0): 0.0, (8, 1): 0.0, (8, 2): 0.0, (8, 3): 0.0, (9, 0): 0.0, (9, 1): 0.0, (9, 2): 0.0, (9, 3): 0.0, (10, 0): 0.0, (10, 1): 0.0, (10, 2): 0.0, (10, 3): 0.0, (11, 0): 0.0, (11, 1): 0.0, (11, 2): 0.0, (11, 3): 0.0, (12, 0): 0.0, (12, 1): 0.0, (12, 2): 0.0, (12, 3): 0.0, (13, 0): 0.0, (13, 1): 0.0, (13, 2): 0.0, (13, 3): 0.0, (14, 0): 0.0, (14, 1): 0.0

## **Part B: Policy Iteration**

- `policy_evaluation(policy)`
- `policy_improvement(policy)`
- `policy_iteration()`
- Print final optimal policy + state values

📌 Flow:
**Initialize policy → Evaluate policy ⟷ Improve policy → Optimal policy**

In [3]:
# ==================================================================================================
# PART B: POLICY ITERATION
# ==================================================================================================
def policy_evaluation(policy):
    v = {}
    for state in range(num_states):
        if state == terminal_state:
            v[state] = 0
        else:
            action = policy[state]
            _, next_state, reward, _ = env.unwrapped.P[state][action][0]
            v[state] = reward + discount_factor * value[next_state]
    return v 

def policy_improvement(policy):
    improved_policy = {}
    for state in range(num_states - 1):  # exclude terminal
        q_values = [compute_q_value(state, action) for action in range(num_actions)]
        max_action = q_values.index(max(q_values))
        improved_policy[state] = max_action
    return improved_policy

def policy_iteration():
    # Start with an arbitrary policy
    policy = {s: 0 for s in range(num_states - 1)}

    while True:
        V = policy_evaluation(policy)
        improved_policy = policy_improvement(policy)

        if improved_policy == policy:
            break
        policy = improved_policy

    return policy, V

# Run policy iteration
policy_pi, V_pi = policy_iteration()
print("Optimal Policy from Policy Iteration:", policy_pi)
print("Optimal State Values from Policy Iteration:", V_pi)

Optimal Policy from Policy Iteration: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 3}
Optimal State Values from Policy Iteration: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 1.0, 15: 0}


## **Part C: Value Iteration**

- `compute_q_value_for_V()`
- `get_max_action_and_values()`
- Value Iteration loop
- Print final optimal policy + values

📌 Flow:
**Initialize V → Bellman updates (value iteration) → Optimal V & Policy**

In [4]:
# ==================================================================================================
# PART C: VALUE ITERATION
# ==================================================================================================
def compute_q_value_for_V(state, action, V):
    if state == terminal_state:
        return 0
    # Access the true transition model from the base env
    _, next_state, reward, _ = env.unwrapped.P[state][action][0]
    return reward + discount_factor * V[next_state]


def get_max_action_and_values(state, V):
    Q_values = [compute_q_value_for_V(state, action, V) for action in range(num_actions)]
    max_action = max(range(num_actions), key=lambda a: Q_values[a])
    max_q_value = Q_values[max_action]
    return max_action, max_q_value

# Initialize
V = {state: 0 for state in range(num_states)}
policy = {state: 0 for state in range(num_states - 1)}
threshold = 0.001

# Value Iteration loop
while True:
    new_V = {state: 0 for state in range(num_states)}

    for state in range(num_states - 1):
        max_action, max_q_value = get_max_action_and_values(state, V)
        new_V[state] = max_q_value
        policy[state] = max_action

    # Convergence check
    if all(abs(new_V[state] - V[state]) < threshold for state in V):
        break

    V = new_V

print("Optimal Policy from Value Iteration:", policy)
print("Optimal State Values from Value Iteration:", V)

Optimal Policy from Value Iteration: {0: 2, 1: 3, 2: 2, 3: 1, 4: 2, 5: 0, 6: 2, 7: 0, 8: 3, 9: 2, 10: 2, 11: 0, 12: 0, 13: 3, 14: 3}
Optimal State Values from Value Iteration: {0: 0.5904900000000002, 1: 0.6561000000000001, 2: 0.7290000000000001, 3: 0.6561000000000001, 4: 0.6561000000000001, 5: 0.0, 6: 0.81, 7: 0.0, 8: 0.7290000000000001, 9: 0.81, 10: 0.9, 11: 0.0, 12: 0.0, 13: 0.9, 14: 1.0, 15: 0}


- Value Iteration provides an efficient alternative to Policy Iteration by combining evaluation and improvement steps.
- While it requires more iterations, each iteration is computationally cheaper, making it particularly suitable for large-scale problems.
- The algorithm is guaranteed to converge to the optimal policy, achieving the same result as Policy Iteration but through a different computational path.

- The choice between `Value Iteration` and `Policy Iteration` depends on the specific problem characteristics: 
  - Use Value Iteration for large problems where computational efficiency per iteration is crucial, 
  - and Policy Iteration for smaller problems where exact solutions and fewer iterations are preferred.